In [1]:
!pip install graphviz
!pip install pyqrlew

You should consider upgrading via the '/Users/victoriadesainteagathe/Documents/codes/DP-SQL/pyqrlew/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/victoriadesainteagathe/Documents/codes/DP-SQL/pyqrlew/myenv/bin/python3 -m pip install --upgrade pip' command.


# Install postgres database

You may skip this section if you have already a postgres database with a connection

In [ ]:
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!service postgresql start

Create a user `alice` with a password 

In [ ]:
!sudo -u postgres psql -c "CREATE USER alice WITH SUPERUSER CREATEDB ENCRYPTED PASSWORD 'my_secret_password';"

Create a databse `mydatabase`

In [ ]:
!sudo -u postgres createdb mydatabase

Create the `census_table` table and push some data on the database. 

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname='mydatabase', user='alice', password='my_secret_password', host='localhost')

# Create a cursor
cursor = conn.cursor()

sql_script = """
CREATE SCHEMA census_schema;

CREATE TABLE census_schema.census_table (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    occupation VARCHAR(50),
    city VARCHAR(50),
    state VARCHAR(50)
);

-- Inserting sample data
INSERT INTO census_schema.census_table (first_name, last_name, age, gender, occupation, city, state)
VALUES
    ('John', 'Doe', 30, 'Male', 'Engineer', 'New York', 'NY'),
    ('Jane', 'Doe', 28, 'Female', 'Doctor', 'Los Angeles', 'CA'),
    ('Bob', 'Smith', 45, 'Male', 'Teacher', 'Chicago', 'IL'),
    ('Alice', 'Johnson', 35, 'Female', 'Artist', 'San Francisco', 'CA'),
    ('Tom', 'Brown', 50, 'Male', 'Lawyer', 'Miami', 'FL'),
    ('Emily', 'Davis', 32, 'Female', 'Software Developer', 'Seattle', 'WA'),
    ('Charlie', 'Miller', 40, 'Male', 'Accountant', 'Boston', 'MA'),
    ('Eva', 'Williams', 29, 'Female', 'Marketing Manager', 'Austin', 'TX'),
    ('Mike', 'Jones', 38, 'Male', 'Chef', 'Denver', 'CO'),
    ('Sophia', 'Taylor', 42, 'Female', 'Business Analyst', 'Atlanta', 'GA');
"""
cursor.execute(sql_script)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

# Use qrlew on the database

In [ ]:
from qrlew_datasets.databases import PostgreSQL

database = PostgreSQL(
    name = "localhost",
    user = "alice",
    password = "my_secret_password",
    port = "5432"
)
ds = database.dataset(
    name = 'census',
    schema_name = 'census_schema',
    ranges=True,
    possible_values_threshold=20,
)
ds.relations()

In [ ]:
import graphviz
display(graphviz.Source(ds.relations()[1][1].dot()))

In [ ]:
privacy_unit = [
    ("census", [], "_PRIVACY_UNIT_ROW_"),
]
synthetic_data = [
    (["extract", "census"], ["extract", "census"]),
]

In [ ]:
import pandas as pd
query = "SELECT sex, COUNT(*) AS my_count FROM extract.census GROUP BY sex ORDER BY sex"
relation = ds.sql(query)
dp_relation = relation.rewrite_with_differential_privacy(
    ds,
    synthetic_data,
    privacy_unit,
    {"epsilon": 1.0, "delta": 5e-4}
).relation()
dp_results = pd.DataFrame(database.eval(dp_relation))
dp_results

In [ ]:
exact_results = pd.DataFrame(database.eval(relation))
exact_results

In [ ]:
dp_relation.render()

In [ ]:
display(graphviz.Source(dp_relation.dot()))